<a href="https://colab.research.google.com/github/KenichiQaz/ML_Learning/blob/main/Own_Try10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *
from IPython.display import display,HTML

     |████████████████████████████████| 719 kB 34.1 MB/s 
     |████████████████████████████████| 365 kB 70.6 MB/s 
     |████████████████████████████████| 4.7 MB 55.8 MB/s 
     |████████████████████████████████| 1.3 MB 56.1 MB/s 
     |████████████████████████████████| 120 kB 68.3 MB/s 
     |████████████████████████████████| 115 kB 75.6 MB/s 
     |████████████████████████████████| 212 kB 58.1 MB/s 
     |████████████████████████████████| 127 kB 71.7 MB/s 
     |████████████████████████████████| 6.6 MB 18.4 MB/s 
Mounted at /content/gdrive


In [2]:
# Word Tokenization with fastai
from fastai.text.all import *
path = untar_data(URLs.IMDB)

In [3]:
#get text files 
files = get_text_files(path, folders = ['train', 'test', 'unsup'])
txt = files[0].open().read(); txt[:75]
# tokenize
spacy = WordTokenizer()
toks = first(spacy([txt]))
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#144) ['A','very','cheesy','and','dull','road','movie',',','with','the','intention','to','be','hip','and','modern',',','shown','in','the','editing','style','and','some','weird','camera','angles',',','resulting','only'...]


# Subword tokenization


In [4]:
txts = L(o.open().read() for o in files[:2000])
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [5]:
# Numericalization with fastai
tkn = Tokenizer(spacy)
toks200 = txts[:200].map(tkn)
num = Numericalize()
num.setup(toks200)
nums = num(toks)[:20]; nums

TensorText([   0,   80, 1456,   13,  341,    0,   26,   10,   32,    9, 1149,   14,   36,  954,   13,  643,   10,  644,   18,    9])

# Preping text for NLP model training

In [6]:
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In [7]:
# Batch 1
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


In [8]:
# Batch 2
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


In [9]:
# Batch 3
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


In [10]:
nums200 = toks200.map(num)
dl = LMDataLoader(nums200)
x,y = first(dl)
x.shape,y.shape
' '.join(num.vocab[o] for o in x[0][:20])
' '.join(num.vocab[o] for o in y[0][:20])

'a very cheesy and dull xxunk movie , with the intention to be hip and modern , shown in the'

In [11]:
# Text Classifier
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj super original . xxmaj really great movie ! xxmaj james xxmaj franco and xxmaj sienna xxmaj miller really do go so well together on screen . i sure hope it comes out soon . i first saw this movie at a screening . xxmaj because of my age and gender , i was asked to stay for a "" talk . "" xxmaj we all got to comment on the movie and a lot of people did n't","xxmaj super original . xxmaj really great movie ! xxmaj james xxmaj franco and xxmaj sienna xxmaj miller really do go so well together on screen . i sure hope it comes out soon . i first saw this movie at a screening . xxmaj because of my age and gender , i was asked to stay for a "" talk . "" xxmaj we all got to comment on the movie and a lot of people did n't seem"
1,"cute xxrep 4 ! xxbos xxmaj if i was the cinematographer who worked on this film , xxmaj i 'd be severely depressed . xxmaj all of that good work in making a colorful , light , and playful world lost in complete ineptitude from everyone else who worked on this movie . \n\n xxmaj the story is actually a good one , despite clichés . a young girl ( 9 years ) named xxmaj ociee xxmaj nash from xxmaj","xxrep 4 ! xxbos xxmaj if i was the cinematographer who worked on this film , xxmaj i 'd be severely depressed . xxmaj all of that good work in making a colorful , light , and playful world lost in complete ineptitude from everyone else who worked on this movie . \n\n xxmaj the story is actually a good one , despite clichés . a young girl ( 9 years ) named xxmaj ociee xxmaj nash from xxmaj mississippi"


# Fine-Tuning the Language Model

In [12]:
# Fine-Tuning the Language Model
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()
learn.fit_one_cycle(1, 2e-2)
# saving model
learn.save('1epoch')

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.998764,3.893725,0.300689,49.093414,27:03


Path('/root/.fastai/data/imdb/models/1epoch.pth')

In [13]:
# Load model
learn = learn.load('1epoch')

In [14]:
#continue training
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)
# save the final encoder
learn.save_encoder('finetuned')

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.751612,3.753862,0.316498,42.685600,28:27
1,3.715760,3.702163,0.322691,40.534878,28:36
2,3.626249,3.649673,0.328548,38.462086,28:42
3,3.566118,3.613624,0.332815,37.100254,28:48
4,3.489405,3.591783,0.335144,36.298740,28:53
5,3.431574,3.576831,0.337712,35.760048,29:02
6,3.368425,3.567200,0.339424,35.417301,29:17
7,3.297986,3.564106,0.340445,35.307880,29:29
8,3.248286,3.567164,0.340656,35.416016,29:18
9,3.219103,3.571495,0.340470,35.569714,29:24


In [15]:
# text generation test
TEXT = "I didn't like this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]
print("\n".join(preds))

i did n't like this movie because i was looking for a good laugh . Unfortunately , I 'm not sure if i can . Being a big Garris fan , i expected this movie to be a fantasy of a father and
i did n't like this movie because i did n't know what to expect . The script was n't as bad as the first movie , but this one has the worst acting i have ever seen . i mean , come on ! Steven


In [16]:
# NLP Datablock
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)
dls_clas.show_batch(max_n=3)
nums_samp = toks200[:10].map(num)
nums_samp.map(len)

,text,category
0,"xxbos xxmaj match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german xxunk by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",pos
1,"xxbos * * attention xxmaj spoilers * * \n\n xxmaj first of all , let me say that xxmaj rob xxmaj roy is one of the best films of the 90 's . xxmaj it was an amazing achievement for all those involved , especially the acting of xxmaj liam xxmaj neeson , xxmaj jessica xxmaj lange , xxmaj john xxmaj hurt , xxmaj brian xxmaj cox , and xxmaj tim xxmaj roth . xxmaj michael xxmaj canton xxmaj jones painted a wonderful portrait of the honor and dishonor that men can represent in themselves . xxmaj but alas … \n\n it constantly , and unfairly gets compared to "" braveheart "" . xxmaj these are two entirely different films , probably only similar in the fact that they are both about xxmaj scots in historical xxmaj scotland . xxmaj yet , this comparison frequently bothers me because it seems",pos
2,"xxbos xxrep 3 * xxmaj warning - this review contains "" plot spoilers , "" though nothing could "" spoil "" this movie any more than it already is . xxmaj it really xxup is that bad . xxrep 3 * \n\n xxmaj before i begin , xxmaj i 'd like to let everyone know that this definitely is one of those so - incredibly - bad - that - you - fall - over - laughing movies . xxmaj if you 're in a lighthearted mood and need a very hearty laugh , this is the movie for you . xxmaj now without further ado , my review : \n\n xxmaj this movie was found in a bargain bin at wal - mart . xxmaj that should be the first clue as to how good of a movie it is . xxmaj secondly , it stars the lame action",neg


(#10) [147,389,183,162,346,79,184,184,236,439]

In [17]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

In [18]:
learn = learn.load_encoder('finetuned')

In [19]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.250436,0.180532,0.930480,01:18


In [20]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.225651,0.165405,0.937480,01:17


In [21]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.185647,0.149014,0.944080,01:34


In [22]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.155531,0.149513,0.945760,01:52
1,0.143802,0.146637,0.946880,01:52


In [23]:
# save the final encoder
learn.save_encoder('finetuned')